In [1]:
import sys
#import cplex
import logging
from configparser import ConfigParser
import uuid
from IPython.display import display_javascript, display_html, display
from IPython.core.display import HTML
import json
config = ConfigParser()
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
modcfg = {}
for nameval in config.items('chenry_MetEngAPI'):
    modcfg[nameval[0]] = nameval[1]
#from chenry_MetEngAPI.chenry_MetEngAPIImpl import chenry_MetEngAPI
from chenry_MetEngAPI.chenry_MetEngAPIServiceClient import chenry_MetEngAPI

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [6]:
impl = chenry_MetEngAPI()
ctx = {}#{"token":"3XJGW6P6P7LOUWNL3IQRUWHX7BOP3WCL"}
output = impl.list_maps({})
output = impl.compute_flux({
    "model":"iML1515",
    "carbon_source":"glc__D_c",
    "target":"THRS",
    "base_media":{
        'glc__D_c':1, 
        'pro__L_e':0.1,
        'ile__L_e':0.1,
        'met__D_e':0.1,
        'lys__L_e':0.1,
        '26dap__M_e':0.1,
        'thm_e':0.1,
        'so4_e':100,
        'pi_e':100,
        'cit_e':0.1,
        'mn2_e': 100,
        'fe2_e': 100,
        'na1_e': 100,
        'mg2_e': 100,
        'nh4_e': 100,
        'k_e': 100,
        'o2_e': 20,
        'h2o_e': 100,
        'h_e': 100,
        'fe3_e': 100.0,
        'zn2_e': 100.0,
        'ca2_e': 100.0,
        'ni2_e': 100.0,
        'cu2_e': 100.0,
        'sel_e': 100.0,
        'cobalt2_e': 100.0,
        'mobd_e': 100.0,
        'cl_e': 100.0,
        'cbl1_e':100,
        'tungs_e': 100.0,
        'slnt_e': 100.0
}})
output = impl.get_map({"model":"iML1515","map_id":"ecoli_cc","reaction_flux":output["reaction_fluxes"]})
f = open("Map.html", "w")
f.write(output["html"])
f.close()
output = impl.get_gene_table_from_model({"model":"iML1515"})
output = impl.get_model_data({"model":"iML1515"})
RenderJSON(output)

In [ ]:
import re
genome = None
ftrhash = {}
with open(modcfg["data_dir"]+"/genomes/iML1515Old.json", 'r') as fh:
    genome = json.load(fh)
for ftr in genome["features"]:
    for alias in ftr["aliases"]:
        if alias[0] == "PATRIC_id":
            ftrhash[alias[1]] = ftr
for ftr in genome["non_coding_features"]:
    for alias in ftr["aliases"]:
        if alias[0] == "PATRIC_id":
            ftrhash[alias[1]] = ftr
gto = {}
with open("/Users/chenry/Downloads/MG1655-core.gto", 'r') as fh:
    gto = json.load(fh)
genecount = len(genome["features"])+len(genome["non_coding_features"])+1
for ftr in gto["features"]:
    if ftr["id"] not in ftrhash:
        if ftr["type"] == "CDS":
            newftr = {
                "aliases": [["PATRIC_id",ftr["id"]]],
                "dna_sequence": "",
                "dna_sequence_length": ftr["location"][0][3],
                "functions": [ftr["function"]],
                "location": ftr["location"],
                "md5": "",
                "ontology_terms": {},
                "protein_md5": "",
                "protein_translation": ftr["protein_translation"],
                "protein_translation_length": len(ftr["protein_translation"]),
            }
            newcds = newftr.copy()
            newftr["type"] = "gene"
            newcds["type"] = "CDS"
            newftr["location"][0][0] = "AYEK01000001"
            newcds["location"][0][0] = "AYEK01000001"
            newftr["id"] = "511145.183_"+str(genecount)
            newcds["id"] = "511145.183_1_CDS_"+str(genecount)
            newftr["cdss"] = [newcds["id"]]
            newcds["parent_gene"] = newftr["id"]
            genome["features"].append(newftr)
            genome["cdss"].append(newcds)
            ftrhash[ftr["id"]] = newftr
            genecount += 1
        else:
            newftr = {
                "aliases": [["PATRIC_id",ftr["id"]]],
                "dna_sequence": "",
                "dna_sequence_length": ftr["location"][0][3],
                "functions": [ftr["function"]],
                "id": "511145.183_"+str(genecount),
                "location": ftr["location"],
                "md5": "",
                "ontology_terms": {},
                "type":ftr["type"]
            }
            newftr["location"][0][0] = "AYEK01000001"
            ftrhash[ftr["id"]] = newftr
            genome["non_coding_features"].append(newftr)
            genecount += 1
    if ftr["id"] in ftrhash:
        for alias in ftr["aliases"]:
            if re.search('^b\d+', alias) != None:
                ftrhash[ftr["id"]]["aliases"].append(["locus",alias])
            elif re.search('^GeneID:', alias) != None:
                ftrhash[ftr["id"]]["aliases"].append(["GeneID",alias[7:]])
            else:
                ftrhash[ftr["id"]]["aliases"].append(["gene_name",alias])
genome_string = json.dumps(genome, indent = 4) 
# Writing to sample.json
with open(modcfg["data_dir"]+"/genomes/iML1515.json", "w") as outfile:
    outfile.write(genome_string)